<a href="https://colab.research.google.com/github/Andjez/Andjez/blob/main/waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q install langchain youtube_transcript_api InstructorEmbedding sentence_transformers faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 120.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.9 MB/s eta 0:

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
import os, re

def extract_video_code(url):
    pattern = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:embed\/|watch\?v=)|youtu\.be\/)([\w-]+)"
    match = re.match(pattern, url)

    if match:
        return match.group(1)
    else:
        return None

# Setup

yt_link = "https://youtu.be/Sgj-Ll9In0o"
if yt_link:
    yt_id = extract_video_code(yt_link)
    transcript = YouTubeTranscriptApi.get_transcript(yt_id)
    my_dict = str(transcript)

In [5]:
print(len(transcript))
print(transcript)

1586
[{'text': "welcome back to unlock Jazz I'm so", 'start': 0.0, 'duration': 4.2}, {'text': 'excited for today because I have Colton', 'start': 2.22, 'duration': 3.9}, {'text': 'Underwood on thank you thanks for having', 'start': 4.2, 'duration': 3.96}, {'text': 'me of course so', 'start': 6.12, 'duration': 4.8}, {'text': 'you were the bachelor obviously you were', 'start': 8.16, 'duration': 4.559}, {'text': "on The Bachelorette weren't you yeah but", 'start': 10.92, 'duration': 3.42}, {'text': "you knew me before all of us I did it's", 'start': 12.719, 'duration': 3.961}, {'text': "not even pretend no so that's what I was", 'start': 14.34, 'duration': 4.56}, {'text': 'gonna go into that but I gotta introduce', 'start': 16.68, 'duration': 4.98}, {'text': 'you first okay yes so', 'start': 18.9, 'duration': 5.219}, {'text': "it's really funny the story", 'start': 21.66, 'duration': 4.56}, {'text': 'actually gotten to speak about this no', 'start': 24.119, 'duration': 5.16}, {'text': "a

In [6]:
data = []
source = []

# Extract 'text' and 'start' values from each dictionary item
for item in transcript:
    data.append(item['text'])
    source.append(item['start'])

# Print the resulting lists
print("data =", data)
print("source =", source)


data = ["welcome back to unlock Jazz I'm so", 'excited for today because I have Colton', 'Underwood on thank you thanks for having', 'me of course so', 'you were the bachelor obviously you were', "on The Bachelorette weren't you yeah but", "you knew me before all of us I did it's", "not even pretend no so that's what I was", 'gonna go into that but I gotta introduce', 'you first okay yes so', "it's really funny the story", 'actually gotten to speak about this no', "and I think it's hilarious because I", 'just have to say when we met the first', 'time like', 'I knew you were gay like I knew it yeah', 'like that was the funny part about it', 'and like you can Emily is sitting right', 'here which is hilarious because Emily', 'was my makeup artist at the time one of', 'my best friends and she was there so and', 'I had my brother as my wingman yes your', 'brother was your wingman and it was the', 'ACM Awards I forgot what year but it was', 'after was after Luke', 'yeah so it was after Luke 

In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = []
metadatas = []
for i, d in enumerate(data):
    splits = text_splitter.split_text(d)
    docs.extend(splits)
    metadatas.extend([{"source": source[i]}] * len(splits))

In [8]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/model"

Mounted at /content/gdrive


In [12]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
store = FAISS.from_texts(data, instructor_embeddings, metadatas=metadatas)


RuntimeError: ignored

In [15]:
faiss.write_index(store.index, "/content/gdrive/MyDrive/model/in_docs.index")
store.index = None
with open("/content/gdrive/MyDrive/model/in_faiss_store.pkl", "wb") as f:
    pickle.dump(store, f)

In [ ]:
  with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
      pickle.dump(vectorStore, f)

In [16]:
!pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 37.5 MB/s eta 0:00:00


In [17]:
from langchain.vectorstores import Chroma, Pinecone
import os, time, pinecone
pinecone_apikey = "bce8df0b-4640-43f3-81f8-22f7a0123aa5"
pinecone_env = "us-west4-gcp-free"
os.environ['PINECONE_API_KEY'] = pinecone_apikey
os.environ['PINECONE_API_ENV'] = pinecone_env

In [18]:
pinecone.init(
    api_key=pinecone_apikey,  # find at app.pinecone.io
    environment=pinecone_env  # next to api key in console
)

In [21]:
#Database
indexlist = pinecone.list_indexes()
index = indexlist[0]

In [22]:
print(index)

new


In [25]:
from sklearn.decomposition import PCA

# Assuming 'embedded_data' is your original embedded data with dimension 768
embedded_data = data  # Your embedded data here

# Define the desired target dimension
target_dimension = 1536

# Perform dimensionality reduction using PCA
pca = PCA(n_components=target_dimension)
reduced_data = pca.fit_transform(embedded_data)

ValueError: ignored

In [24]:
Pinecone.from_texts(data, instructor_embeddings, metadatas=metadatas, index_name=index)

ApiException: ignored